## Libraries
This notebook explains how to retrieve data for and edit private and public ADS libraries. Examples here will be shown using Python with the use of the [requests](http://docs.python-requests.org/en/master/) library, though the same work could be done using the [unofficial Python ADS library](https://ads.readthedocs.io/en/latest/) or curl commands on the command line (see the "API documentation - UNIX shell" folder in this same repository).

The base URL for library queries is
```
https://api.adsabs.harvard.edu/v1/biblib/<endpoint>
```
where the allowed `<endpoint>` values are:
* `/libraries`: create or view all libraries for a given user; GET or POST methods allowed
* `/libraries/<library_id>`: view a specific library; GET method is allowed
* `/documents/<library_id>`: add and remove records from a specific library, update the metadata for a specific library, or delete an entire library; POST, PUT, and DELETE methods are allowed
* `/query/<library_id>`: add and remove records from a specific library based on the results of a given search query; POST and GET methods are allowed
* `/permissions/<library_id>`: view or modify the permissions for a specific library; GET and POST methods are allowed
* `/transfer/<library_id>`: transfer ownership of a library to another ADS user; GET and POST methods are allowed

These endpoints are discussed further below, with examples.

In all examples below, `token` should be replaced with [your own API token](https://ui.adsabs.harvard.edu/user/settings/token). If you haven't worked with our API before, it's recommended that you read the [README](https://github.com/adsabs/adsabs-dev-api/blob/master/README.md) before beginning.

In [1]:
# import the requests package and set your token in a variable for later use
import requests

token="your-token-here"

### Permissions
  * **owner**: Owns the library. Can read libraries, add/remove bibcodes, delete the library, add/remove permissions, transfer ownership of the library.
  * **admin**: Can read libraries, add/remove bibcodes, add/remove permissions.
  * **write**: Can read libraries, add/remove bibcdes.
  * **read**: Can read libraries.

Any other user that does not have a permission to the library they try to visit, can only see it if it is public. If the user has no permissions, they cannot view this library.

### `/libraries`
Create a library or view all libraries for a given user, depending on the method used.

#### GET method
View all libraries for the user associated with the given API token.

The number of libraries can be controlled by specifiying the `start` and `rows` params.

*Return data*:
  * name: (string) name of the library
  * id: (string) ID of the library
  * description: (string) description of the library
  * num_documents: (integer) number of documents in the library
  * date_created: (string) ISO date library was created
  * date_last_modified: (string) ISO date library was last modified
  * permission: (string) permission type, can be: 'read', 'write', 'admin', or 'owner'
  * public: (boolean) true means it is a public library
  * num_users: (integer) number of users with permissions to this library
  * owner: (string) ID of the user who created the library

In [2]:
results = requests.get("https://api.adsabs.harvard.edu/v1/biblib/libraries",
                       headers={'Authorization': 'Bearer ' + token})
len(results.json()['libraries'])

11

In [3]:
results = requests.get("https://api.adsabs.harvard.edu/v1/biblib/libraries/?start=0&rows=11",
                       headers={'Authorization': 'Bearer ' + token})
len(results.json()['libraries'])

11

#### POST method
Create a library for the requesting user. Returns the metadata of the library created.

In addition to the usual token header and request URL, you must specify that this is a POST request. The POST payload must also be passed using the `data` keyword and should be in JSON format. We use the `json` library here to convert between Python dictionaries and JSON strings. The payload should include the following keys:
  * name: (string) optional; name of the library (must be unique for that user). The default is `Untitled Library`, with an incremented integer starting with 1 appended
  * description: (string) optional; description of the library. The default is "My ADS library"
  * public: (boolean) optional; set to true to allow public access. Note: the Boolean value is case senstitive, and must be entered in lowercase. The default is false, or a private library
  * bibcode: (list) optional; list of bibcodes to add. The default is an empty library

The metadata of the library you just created will be returned after the request:
  * name: (string) name of the library
  * id: (string) ID of the library
  * description: (string) description of the library

In [3]:
import json

payload = {"name": "API Library", 
           "description": "This library is created via API", 
           "public": True, 
           "bibcode": ["1975CMaPh..43..199H", "1973PhRvD...7.2333B"]}
results = requests.post("https://api.adsabs.harvard.edu/v1/biblib/libraries",
                        headers={'Authorization': 'Bearer ' + token}, 
                        data=json.dumps(payload))
results.json()

{'name': 'API Library',
 'id': 'Z7cCXDXQRC2HxB8CA8uu2g',
 'description': 'This library is created via API',
 'bibcode': ['1975CMaPh..43..199H', '1973PhRvD...7.2333B']}

### /libraries/&lt;library_id&gt;
View a specific library. The `library_id` (string) is the unique identifier of the library, given via the `/libraries` GET method detailed above.

To edit or delete a library, see the `/document/<library_id>` endpoint section below.

#### GET
Obtains all the documents in a given library. 

*Return data*:
  * documents: (list) a list containing the bibcodes in the library
  * solr: (dict) the response from the solr bigquery endpoint
  * metadata: (dict) Contains the following:
        * name: (string) name of the library
        * id: (string) ID of the library
        * description: (string) description of the library
        * num_documents: (integer) number of documents in the library
        * date_created: (string) ISO date library was created
        * date_last_modified: (string) ISO date library was last modified
        * permission: (sting) Permission type, can be 'read', 'write', 'admin', or 'owner'
        * public: (boolean) true means it is public
        * num_users: (integer) number of users with permissions to this library
        * owner: (string) identifier of the user who created the library
  * updates: (dict) Contains the following
        * num_updated: (integer) number of documents modified based on the response from solr
        * duplicates_removed: (integer) number of files removed because they are duplications
        * update_list: (list, dict) list of dictionaries such that a single element described the original bibcode (key) and the updated bibcode now being stored (item)

The following type of user can read a library:
  - owner
  - admin
  - write
  - read

In [4]:
# get the data for the library created above
results = requests.get("https://api.adsabs.harvard.edu/v1/biblib/libraries/Z7cCXDXQRC2HxB8CA8uu2g",
                       headers={'Authorization': 'Bearer ' + token})
results.json()

{'documents': ['1975CMaPh..43..199H', '1973PhRvD...7.2333B'],
 'solr': {'responseHeader': {'status': 0,
   'QTime': 34,
   'params': {'q': '*:*',
    'fl': 'bibcode,alternate_bibcode',
    'start': '0',
    'internal_logging_params': 'X-Amzn-Trace-Id=Root=1-64011d99-0652bc5e7903b9374dff1eee',
    'fq': '{!bitset}',
    'sort': 'date desc',
    'rows': '20',
    'wt': 'json'}},
  'response': {'numFound': 2,
   'start': 0,
   'docs': [{'bibcode': '1975CMaPh..43..199H'},
    {'bibcode': '1973PhRvD...7.2333B'}]}},
 'metadata': {'name': 'API Library',
  'id': 'Z7cCXDXQRC2HxB8CA8uu2g',
  'description': 'This library is created via API',
  'num_documents': 2,
  'date_created': '2023-03-02T22:05:01.292494',
  'date_last_modified': '2023-03-02T22:05:01.292502',
  'permission': 'owner',
  'public': True,
  'num_users': 1,
  'owner': 'kelly.lockhart'},
 'updates': {'num_updated': 0, 'duplicates_removed': 0, 'update_list': []}}

### /libraries/operations/&lt;library_id&gt;
Perform set operations on one or more libraries. The `library_id` (string) is the unique identifier of the primary library, as specified in the `/libraries` GET response. Depending on the operation, a secondary library ID may need to be passed in the post payload.

Function allowed: POST

#### POST
The following operations are supported:
* union: take the union of the primary and the secondary libraries. The result is saved to a new library
* intersection: take the intersection of the primary and the secondary libraries. The result is saved to a new library
* difference: take the difference between the primary and the secondary libraries. The result is saved to a new library
* copy: copy the contents of the primary library into the secondary library. The secondary library is not emptied first; use the empty operation on the secondary library first in order to create a duplicate of the primary library in the secondary library
* empty: empty the primary library of its contents (no secondary library ID is needed)

Specify that the request is a POST method. You must include a payload, passed via the `data` keyword, in JSON format. The payload must include:
* action: (string, required) set operation to perform; allowed values are [`union`, `intersection`, `difference`, `copy`, `empty`]
* libraries: (list) list of secondary library IDs; multiple secondary libraries are allowed for [`union`, `intersection`, `difference`]; one secondary library is allowed for `copy`; no secondary libraries are allowed for `empty`
* name: (string, optional) name of the new library to be created for [`union`, `intersection`, `difference`]; the name must be unique for a given user; if no name is specified, the name used will be "Untitled" with a timestamp
* description: (string, optional) description of the new library to be created for [`union`, `intersection`, `difference`]; if no description is specified, the description used will include the set operation used and the primary and secondary library IDs
* public: (Boolean, optional) sets whether the new library created by [`union`, `intersection`, `difference`] is publicly viewable or not; the new library will be private unless otherwise specified

*Return data*:
* name: (string) Name of the library
* id: (string) ID of the library
* description: (string) Description of the library

*Permissions*:
The following type of user can perform set operations:
* owner
* admin
* write


In [5]:
# take the union of 2 libraries, using the default description and public settings
payload = {"action": "union", 
           "libraries": ["s56fjycaRMKxw6tDEr31XQ"], 
           "name": "New union library"}
results = requests.post("https://api.adsabs.harvard.edu/v1/biblib/libraries/operations/iRZhyhvaSWe0uWwcXKt77w", 
                        headers={'Authorization': 'Bearer ' + token},
                        data=json.dumps(payload))
results.json()

{'name': 'New union library',
 'id': 'M74cJpN4SXuunD4X7zorpw',
 'description': 'Union of libraries Merger Shocks, Metal Grads (IDs: iRZhyhvaSWe0uWwcXKt77w, s56fjycaRMKxw6tDEr31XQ)',
 'bibcode': ['2010ApJ...721L..48K',
  '2010ASPC..423..355R',
  '2011ApJ...734...87R',
  '2010ApJ...723.1255R',
  '2010ApJ...710L.156R',
  '2010ApJ...721..505R']}

In [6]:
# take the intersection of 2 libraries, using the default public settings
payload = {"action": "intersection", 
           "libraries": ["YhrBmqfRRd22j-CeBn_1sg"], 
           "name": "New intersection library", 
           "description": "Sample intersection library"}
results = requests.post("https://api.adsabs.harvard.edu/v1/biblib/libraries/operations/Z7cCXDXQRC2HxB8CA8uu2g", 
                        headers={'Authorization': 'Bearer ' + token},
                        data=json.dumps(payload))
results.json()

{'name': 'New intersection library',
 'id': 'h1j8YE_hTvSVZT7rWhrDXQ',
 'description': 'Sample intersection library',
 'bibcode': ['1975CMaPh..43..199H']}

In [7]:
# take the difference of 2 libraries
payload = {"action": "difference", 
           "libraries": ["Z7cCXDXQRC2HxB8CA8uu2g"], 
           "name": "New difference library", 
           "description": "Sample difference library", 
           "public": True}
results = requests.post("https://api.adsabs.harvard.edu/v1/biblib/libraries/operations/YhrBmqfRRd22j-CeBn_1sg", 
                        headers={'Authorization': 'Bearer ' + token},
                        data=json.dumps(payload))
results.json()

{'name': 'New difference library',
 'id': 'A2Mstu9XTy-yzh48oPNrFQ',
 'description': 'Sample difference library',
 'bibcode': ['2014AJ....147..124M',
  '2009ApJ...706..797T',
  '2012yCat..21990026H',
  '2010AJ....139.2620N',
  '2009MNRAS.399..683J',
  '2011ASSP...24...11H',
  '2010CBET.2338....2N',
  '2012yCat.2281....0C',
  '2010yCat.7259....0J',
  '2009AAS...21360808T',
  '2011AJ....142..183N',
  '2010A&A...511A..23P',
  '2011AAS...21715002C',
  '2009ApJ...700..331H',
  '2011MNRAS.413.2906D',
  '2012ApJS..199...26H',
  '2012yCat..51392620N',
  '2010noao.prop..445P',
  '2010yCat..73990683J',
  '2009AJ....138..547S',
  '2010AJ....139.1413N',
  '2012ApJ...759....6E',
  '2020AAS...23528705A',
  '2010PASA...27..302M',
  '2010AJ....139.1178N',
  '2009AJ....138.1667B']}

In [8]:
# empty a library of all of its bibcode contents
payload = {"action": "empty"}
results = requests.post("https://api.adsabs.harvard.edu/v1/biblib/libraries/operations/h1j8YE_hTvSVZT7rWhrDXQ",
                        headers={'Authorization': 'Bearer ' + token},
                        data=json.dumps(payload))
results.json()

{'name': 'New intersection library',
 'description': 'Sample intersection library',
 'public': False,
 'bibcode': []}

In [9]:
# copy the bibcode contents from the primary library to the secondary library (this will not empty the library first)
payload = {"action": "copy", 
           "libraries": ["h1j8YE_hTvSVZT7rWhrDXQ"]}
results = requests.post("https://api.adsabs.harvard.edu/v1/biblib/libraries/operations/iRZhyhvaSWe0uWwcXKt77w",
                        headers={'Authorization': 'Bearer ' + token},
                        data=json.dumps(payload))
results.json()

{'name': 'New intersection library',
 'description': 'Sample intersection library',
 'public': False,
 'bibcode': ['2011ApJ...734...87R', '2010ApJ...721..505R']}

### /documents/&lt;library_id&gt;
Interact with a specific library. The `library_id` (string) is the unique identifier of the library, as specified in the `/libraries` GET response.

Functions allowed: add records to a library (POST), delete records from a library (POST), update the metadata of a library (PUT), delete the entire library (DELETE)

#### POST
Add or delete a document to a given library. Returns the number of documents added or removed

Specify that the request is a POST method on the request. You must include a payload, passed via the `data` keyword, in JSON format. The payload must include:
  * bibcode: (list) list of bibcodes to be added or removed
  * action: (string) 'add' or 'remove' to add or remove the given bibcodes from the specified library

*Return data*:
  * number_added: (integer) number of documents added (if 'add' is used)
  * number_removed: (integer) number of documents removed (if 'remove' is used)
  * invalid_bibcodes: (list) List of any specified bibcodes that were not found in ADS

*Permissions*:
The following type of user can add or remove documents:
  * owner
  * admin
  * write

In [10]:
# add a bibcode
payload = {"bibcode": ["1974Natur.248...30H"], "action": "add"}
results = requests.post("https://api.adsabs.harvard.edu/v1/biblib/documents/h1j8YE_hTvSVZT7rWhrDXQ", 
                        headers={'Authorization': 'Bearer ' + token},
                        data=json.dumps(payload))
results.json()

{'number_added': 1}

In [11]:
# remove it again
payload = {"bibcode": ["1974Natur.248...30H"], "action": "remove"}
results = requests.post("https://api.adsabs.harvard.edu/v1/biblib/documents/h1j8YE_hTvSVZT7rWhrDXQ", 
                        headers={'Authorization': 'Bearer ' + token},
                        data=json.dumps(payload))
results.json()

{'number_removed': 1}

#### PUT
Update the metadata of a given library. Returns updated metadata of the library

As with the POST method, you must specify that the request is a PUT method. You must also include a payload, passed via the `data` keyword, in JSON format. The payload can include:

*Payload*:
  * name: (string) name of the library
  * description: (string) description of the library
  * public: (boolean) true if the library should be publicly viewable, false if it should be private (note that this is case sensitive and should be lower case)

*Note: The above are optional. Leave fields out if they do not need to be updated.*

*Return data*:
  * name: (string) name of the library
  * description: (string) description of the library
  * public:(boolean) true if the library is publicly viewable, false if it is private

*Note: returns the key/value that was requested to be updated*

*Permissions*:
The following type of user can update the 'name', 'library', and 'public' attributes:
  - owner
  - admin

In [12]:
# change the metadata of a library
payload = {"name": "New API library name", 
           "description": "New API library description", 
           "public": False}
results = requests.put("https://api.adsabs.harvard.edu/v1/biblib/documents/h1j8YE_hTvSVZT7rWhrDXQ",
                       headers={'Authorization': 'Bearer ' + token},
                       data=json.dumps(payload))
results.json()

{'name': 'New API library name',
 'description': 'New API library description',
 'public': False}

##### DELETE
Delete the given library

Make sure to specify the DELETE method. No payload is required, as the library ID is passed in the URL. There is no return response to the request.

*Permissions*: The following type of user can update a library:
  - owner

In [13]:
# create a temporary library
payload = {"name":"temp library"}
results = requests.post("https://api.adsabs.harvard.edu/v1/biblib/libraries",
                        headers={'Authorization': 'Bearer ' + token},
                        data=json.dumps(payload))
results.json()

{'name': 'temp library',
 'id': 'LognylxoS5WodJuQaldUQA',
 'description': 'My ADS library'}

In [14]:
# delete the temporary library
results = requests.delete("https://api.adsabs.harvard.edu/v1/biblib/documents/LognylxoS5WodJuQaldUQA",
                         headers={'Authorization': 'Bearer ' + token})

results.json()

{}

### /query/&lt;library_id&gt;
Add or remove documents from the library specified by `library_id` (string), where `library_id` is the unique identifier of the library, as specified in the `/libraries` GET response.

Functions allowed: add query to a library (POST, GET), delete query from a library (POST).

#### POST
Add or delete a query to/from a given library. Returns the number of documents added or removed along with the added or removed bibcodes.

Specify that the request is a POST method. You must include a payload, passed via the `data` keyword, in JSON format. The payload must include:
  * query: (json) dict of query parameters (see Search notebook for more details and example queries.)
  * action: (string) 'add' or 'remove' to add or remove the given bibcodes from the specified library

*Return data*:
  * number_added: (integer) number of documents added (if 'add' is used)
  * number_removed: (integer) number of documents removed (if 'remove' is used)
  * bibcodes: (list) list of bibcodes added to/removed from the library

*Permissions*:
The following type of user can add or remove documents:
  * owner
  * admin
  * write


In [15]:
# add a query by posting search parameters
payload = {"params": {"q": "black holes", 
                      "fq": "database:astronomy"}, 
           "action": "add"}
results = requests.post("https://api.adsabs.harvard.edu/v1/biblib/query/MaFX6tuGSaWumxjrm4ri2A",
                        headers={'Authorization': 'Bearer ' + token},
                        data=json.dumps(payload))

In [16]:
# remove the bibcode by query
payload = {"params": {"q": "black holes", 
                      "fq": "database:astronomy"}, 
           "action": "remove"}
results = requests.post("https://api.adsabs.harvard.edu/v1/biblib/query/MaFX6tuGSaWumxjrm4ri2A",
                        headers={'Authorization': 'Bearer ' + token},
                        data=json.dumps(payload))

#### GET
Add a document to a given library. Returns the number of documents added along with a list of added bibcodes

The `GET` request for this endpoint can only be used to add documents to the library. For removal by query, see the `POST` request above.

  * Query of the form:  `<biblib_API_url>/query/<library_id>?q=<query>&<additional_params>` 
    * See the Search notebook for more details and query examples.
    * It is important to note that there is no `/` between the `library_id` and the query string (`?q=<query>`).

*Return data*:
  * number_added: (integer) number of documents added
  * bibcodes: (list) list of added bibcodes

*Permissions*:
The following type of user can add or remove documents:
  * owner
  * admin
  * write

In [17]:
# add a query with a GET request
results = requests.get("https://api.adsabs.harvard.edu/v1/biblib/query/MaFX6tuGSaWumxjrm4ri2A?q=black+holes&fq=database:astronomy",
                       headers={'Authorization': 'Bearer ' + token})

### /permissions/&lt;library_id&gt;
Edit the permissions of a library. The `library_id` (string) is the unique identifier of the library, as specified in the `/libraries` GET response.

#### GET
Get the permissions for a given library.

The response is a list of dictionaries, where each dictionary is for a specific user. The dictionary contains a list of all the permissions of the user:
  	* user dictionary (dict) contains:
  		* key (string): user e-mail
  		* item (list): list of permissions ('read', 'write', 'admin', 'owner')

*Permissions*:
The following type of user can access permissions of a library:
  - owner
  - admin

In [18]:
# get the library permissions
results = requests.get("https://api.adsabs.harvard.edu/v1/biblib/permissions/h1j8YE_hTvSVZT7rWhrDXQ",
                       headers={'Authorization': 'Bearer ' + token})
results.json()

[{'kelly.lockhart@cfa.harvard.edu': ['owner']}]

#### POST
Edit the permissions of a library.

Specify that the request is a POST method. You must include a payload, passed via the `data` keyword, in JSON format. The payload must include:
  * email: (string) specifies which user's permissions to be modified
  * permission: (dict) specifies the permissions type and whether the user has this permission
   - allowed keys: (string) 'read', 'write', or 'admin'
   - allowed values: (boolean) whether the user has this permission (true=yes, false=no)

There is no response to the request.

Note: An email will be sent to the affected user upon a successful permissions change.

*Permissions*:
The following type of user can update a permission for a user for a given library:
  - owner
  - admin

In [19]:
# edit the library permissions
payload = {"email":"test@email.com", 
           "permission": {"read": True}}
results = requests.post("https://api.adsabs.harvard.edu/v1/biblib/permissions/h1j8YE_hTvSVZT7rWhrDXQ",
                        headers={'Authorization': 'Bearer ' + token},
                        data=json.dumps(payload))
results.json()

{}

### /transfer/&lt;library_id&gt;
Transfer a the ownership of a library to another ADS user. The `library_id` (string) is the unique identifier of the library, as specified in the `/libraries` GET response.

#### POST
Transfer the ownership of a library from one user to another. 

Specify that the request is a POST method. You must include a payload, passed via the `data` keyword, in JSON format. The payload must include:
  * email: (string) email of the user the account should be transfered to

There is no response to the request.

Note: An email will be sent to the affected user upon a successful permissions change.

*Permissions*:
The following type of user can transfer libraries:
  - owner

In [20]:
# transfer the library ownership
payload = {"email": "test@email.com"}
results = requests.post("https://api.adsabs.harvard.edu/v1/biblib/transfer/h1j8YE_hTvSVZT7rWhrDXQ",
                        headers={'Authorization': 'Bearer ' + token},
                        data=json.dumps(payload))
results.json()

{}